In [4]:
import tweepy
import json
import csv

In [5]:
#Twitter API credentials
consumer_key = 'pe7gsS8WNkANobhPvKU5q9PPv'
consumer_secret = 'EF8F0wezQz8HPBRp17DMXTEBrxTXJLBs1mzPnzjLLDlYsHql7D'
access_token = '468248438-aR7mCZ40MKTFq2TNIOGN0IWEegXUBGKDpN3QtDyT'
access_token_secret = 'K5dXjiphLnk17QfWTxigJHOJjgM3005T3c2OZj2X79FA8'

In [8]:
#boundingbox ciudad de Madrid obtenido de https://boundingbox.klokantech.com/
madrid = [-3.7475842804,40.3721683069,-3.6409114868,40.4886258195] 

#para escribir sobre un txt (tomar decision si csv o database)

class MyStreamListener(tweepy.StreamListener):


    def on_data(self,data):
        # Twitter returns data in JSON format - we need to decode it first
        try:
            decoded = json.loads(data)
            
        except Exception as e:
            print ("Error on_data: %s" % str(e)) #we don't want the listener to stop
            return True
        print(decoded)
        
        #Si es localizacion exacta
        if decoded.get('geo') is not None:
            coord = decoded.get('geo').get('coordinates')
        else:
            coord = 'No found'
        
        
        user_name = '@' + decoded.get('user').get('screen_name') #nombre cuenta @itdUPM 
        user_id=decoded.get('user').get('id') #id de la cuenta (int)
        
        created_at = decoded.get('created_at') #Fecha
        text = decoded['text'].replace('\n',' ') #Contenido tweet
        tweet_id = decoded['id'] #tweet id (int64)
        source = decoded['source'] #string source (web client, android, iphone) interesante???
        
        reply_count = decoded['reply_count'] #Number of times this Tweet has been replied to
        retweet_count = decoded['retweet_count'] #Number of times this Tweet has been retweeted
        favorite_count = decoded['favorite_count'] #how many times this Tweet has been liked by Twitter users.
        
        #in_reply_to_status_id  Si es una respuesta id del tweet al que responde
        #in_reply_to_user_id    Si es una respuesta user_id al que responde
        #in_reply_to_screen_name Si es una respuesta screen_name al que responde
        
        tweet = ['%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s\n' % (tweet_id,user_name,user_id,created_at,text,source,
                                      reply_count,retweet_count,favorite_count,
                                      place_type,place_name,coord )]
        
        # Writing tweets data
        with open('OutputStreamingGeoLocated.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(tweet)
             
        print(tweet)
        
        
        return True
        
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

    # returning non-False reconnects the stream, with backoff.


if __name__ == '__main__':
    print ('Starting')
    #authorize twitter, initialize tweepy        
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    
    #create the api and the stream object
    myStreamListener = MyStreamListener()
    myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
    #Filter the stream by keywords
    myStream.filter(locations = madrid)

with open('OutputStreamingGeoLocated.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['tweet_id','user_name','user_id','created_at','text','source',
                                      'reply_count','retweet_count','favorite_count',
                                      'place_type','place_name','coord'])

Starting
